<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Metodo_de_composicion_musical_vs_Alpine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def alpine(x, y):
    """Calcula la funcion Alpine en 2D."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

## Inicialización del compositor

In [ ]:
def crear_compositor(melodias_por_comp, rango_x, rango_y):
    """
    Crea un compositor para MMC generando melodias iniciales
    dentro del espacio de busqueda.
    """

    melodias = []
    valores = []

    # Generar melodias iniciales aleatorias
    for _ in range(melodias_por_comp):
        x = random.uniform(rango_x[0], rango_x[1])
        y = random.uniform(rango_y[0], rango_y[1])
        melodia = (x, y)

        # Evaluar la melodia
        valor = alpine(x, y)

        melodias.append(melodia)
        valores.append(valor)

    # Identificar la mejor melodia inicial
    indice_mejor = np.argmin(valores)
    mejor_melodia = melodias[indice_mejor]
    mejor_valor = valores[indice_mejor]

    # Crear estructura del compositor
    compositor = {
        "melodias": melodias,
        "valores": valores,
        "mejor_melodia": mejor_melodia,
        "mejor_valor": mejor_valor
    }

    return compositor

## Generación de nuevas melodías

In [ ]:
def generar_melodia(compositor, mejor_vecino,
                    rango_x, rango_y, alpha=0.5, beta=0.5):
    """
    Genera una nueva melodia combinando:
    - mejor melodia propia,
    - mejor melodia del vecino,
    - perturbacion aleatoria.
    """

    # Extraer mejor melodia propia
    x_propio, y_propio = compositor["mejor_melodia"]

    # Extraer mejor melodia del vecino
    x_vecino, y_vecino = mejor_vecino

    # Combinacion lineal segun la ecuacion de MMC
    x_combinado = alpha * x_propio + beta * x_vecino
    y_combinado = alpha * y_propio + beta * y_vecino


    # Magnitud de perturbacion: 10% del rango de cada eje
    magnitud_x = 0.1 * (rango_x[1] - rango_x[0])
    magnitud_y = 0.1 * (rango_y[1] - rango_y[0])

    # Agregar perturbacion uniforme
    x_nueva = x_combinado + random.uniform(-magnitud_x, magnitud_x)
    y_nueva = y_combinado + random.uniform(-magnitud_y, magnitud_y)

    # Ajustar al borde mas cercano si se sale del rango
    x_nueva = np.clip(x_nueva, rango_x[0], rango_x[1])
    y_nueva = np.clip(y_nueva, rango_y[0], rango_y[1])

    return (x_nueva, y_nueva)

## Actualización de la matriz de partituras

In [ ]:
def actualizar_partituras(compositor, melodia_nueva, valor_nuevo):
    """
    Actualiza la matriz de partituras del compositor.
    Reemplaza a la peor melodia si la nueva es mejor.
    """
    valores = compositor["valores"]
    melodias = compositor["melodias"]

    # Encontrar el peor valor (maximo en minimizacion)
    indice_peor = np.argmax(valores)
    valor_peor = valores[indice_peor]

    # Reemplazo elitista: solo aceptar si mejora
    if valor_nuevo < valor_peor:
        melodias[indice_peor] = melodia_nueva
        valores[indice_peor] = valor_nuevo

        # Actualizar mejor melodia personal
        if valor_nuevo < compositor["mejor_valor"]:
            compositor["mejor_valor"] = valor_nuevo
            compositor["mejor_melodia"] = melodia_nueva

Ejemplo de uso

In [ ]:
# Crear un compositor con 5 melodias iniciales
compositor = crear_compositor(
    melodias_por_comp=5,
    rango_x=(-5, 5),
    rango_y=(-5, 5)
)

print("Estado inicial del compositor:")
print("  Mejor melodia:", compositor["mejor_melodia"])
print("  Mejor valor:", compositor["mejor_valor"])
print("  Total de melodias:", len(compositor["melodias"]))

# Melodia de un vecino (influencia social)
vecino_melodia = (0.5, 0.5)

# Generar melodia candidata
candidata = generar_melodia(
    compositor,
    mejor_vecino=vecino_melodia,
    rango_x=(-5, 5),
    rango_y=(-5, 5),
    alpha=0.5,
    beta=0.5
)

valor_candidata = alpine(*candidata)
print("\nMelodia candidata:", candidata)
print("Valor:", valor_candidata)

# Actualizar la matriz si corresponde
actualizar_partituras(compositor, candidata, valor_candidata)

# Verificacion rapida
if valor_candidata <= compositor["mejor_valor"] + 1e-6:
    print("La candidata mejoro la mejor melodia personal!")



# Función principal de MMC

## Inicialización de la sociedad

In [ ]:
def mmc(max_iter, num_compositores, melodias_por_comp,
        alpha=0.5, beta=0.5, rango_x=(-5, 5), rango_y=(-5, 5)):
    """Ejecuta el Metodo de Composicion Musical (MMC) sobre la funcion Alpine."""

    # Crear la sociedad inicial de compositores
    compositores = [
        crear_compositor(melodias_por_comp, rango_x, rango_y)
        for _ in range(num_compositores)
    ]

    # Mejor solucion global inicial (gbest)
    mejor_global = min(
        compositores,
        key=lambda c: c["mejor_valor"]
    )
    gbest = mejor_global["mejor_melodia"]
    gbest_valor = mejor_global["mejor_valor"]

    # Historial de valores para analizar la convergencia
    historial_valores = [gbest_valor]

    # Guardar coordenadas para visualizacion (opcional)
    coordenadas_x = [
        np.array([c["mejor_melodia"][0] for c in compositores])
    ]
    coordenadas_y = [
        np.array([c["mejor_melodia"][1] for c in compositores])
    ]

    # Ciclo principal de MMC
    for iteracion in range(max_iter):

        # Cada compositor genera y evalúa una nueva melodía
        for i, compositor in enumerate(compositores):

            # Topología completa: elegir un vecino aleatorio distinto
            indices_vecinos = [j for j in range(num_compositores) if j != i]
            indice_vecino = random.choice(indices_vecinos)
            vecino = compositores[indice_vecino]

            # Obtener la mejor melodía del vecino
            mejor_vecino = vecino["mejor_melodia"]

            # Generar nueva melodía candidata
            candidata = generar_melodia(
                compositor,
                mejor_vecino=mejor_vecino,
                alpha=alpha,
                beta=beta,
                rango_x=rango_x,
                rango_y=rango_y
            )

            # Evaluar la candidata con la función objetivo
            x_cand, y_cand = candidata
            valor_cand = alpine(x_cand, y_cand)
            # Actualizar matriz de partituras (elitismo)
            actualizar_partituras(compositor, candidata, valor_cand)

        # Actualizar la mejor solución global si corresponde
        if valor_cand < gbest_valor:
            gbest = candidata
            gbest_valor = valor_cand

        # Guardar historial para convergencia y visualización (opcional)
        historial_valores.append(gbest_valor)
        coordenadas_x.append(np.array([c["mejor_melodia"][0] for c in compositores]))
        coordenadas_y.append(np.array([c["mejor_melodia"][1] for c in compositores]))

    # Devolver resultados al completar todas las iteraciones
    x_opt, y_opt = gbest
    return (x_opt, y_opt, gbest_valor,
            historial_valores, coordenadas_x, coordenadas_y)

## Configuración y ejecución

In [ ]:
# === Configuracion de parametros ===
max_iter = 50
num_compositores = 10
melodias_por_comp = 5
alpha = 0.1  # Peso de influencia propia
beta = 0.1   # Peso de influencia social

print("="*60)
print("CONFIGURACION DE MMC")
print("="*60)
print(f"Numero de compositores: {num_compositores}")
print(f"Melodias por compositor: {melodias_por_comp}")
print(f"Iteraciones maximas: {max_iter}")
print(f"Alpha (influencia propia): {alpha}")
print(f"Beta (influencia social): {beta}")
print(f"Evaluaciones totales: {num_compositores * max_iter}")
print()

# === Ejecutar MMC ===
x_opt, y_opt, valor_opt, historial, coord_x, coord_y = mmc(
    max_iter=max_iter,
    num_compositores=num_compositores,
    melodias_por_comp=melodias_por_comp,
    alpha=alpha,
    beta=beta
)

# === Imprimir resultados ===
print("="*60)
print("RESULTADOS DE MMC")
print("="*60)
print(f"Mejor solucion encontrada:")
print(f"  X = {x_opt:.6f}")
print(f"  Y = {y_opt:.6f}")
print(f"  Valor Alpine = {valor_opt:.6f}")
print()
print(f"Optimo teorico: f(0, 0) = 0")
print(f"Distancia al optimo: {np.sqrt(x_opt**2 + y_opt**2):.6f}")

## Veamos una gráfica de convergencia

In [ ]:
# === Crear grafica de convergencia ===
plt.figure(figsize=(10, 6))
iteraciones = range(len(historial))

plt.plot(iteraciones, historial, 'b-', linewidth=2, label='Mejor valor')
plt.scatter(iteraciones, historial, c='red', s=30, zorder=5)

plt.xlabel('Generacion', fontsize=12)
plt.ylabel('Mejor valor encontrado', fontsize=12)
plt.title('Convergencia de MMC en funcion Alpine',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Veamos la trayectoria de un individuo

In [ ]:
# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura ===
plt.figure(figsize=(10, 8))

# Graficar curvas de nivel (topografia de Alpine)
plt.contour(X, Y, Z, levels=20, colors='gray', alpha=0.5)
contourf = plt.contourf(X, Y, Z, levels=30, cmap='viridis', alpha=0.6)
plt.colorbar(contourf, label='Valor de Alpine')

# === Seleccionar compositor a visualizar ===
compositor_id = 0  # Compositor 0 (puedes cambiar a 1, 2, ..., 9)

# === Extraer trayectoria completa del compositor ===
trayectoria_x = [coord_x[t][compositor_id] for t in range(len(coord_x))]
trayectoria_y = [coord_y[t][compositor_id] for t in range(len(coord_y))]

# === Graficar trayectoria con linea y marcadores ===
plt.plot(
    trayectoria_x, trayectoria_y,
    color='red',
    marker='o',
    markersize=6,
    linestyle='-',
    linewidth=2,
    label=f'Compositor {compositor_id}',
    alpha=0.8
)

# Marcar inicio con circulo verde grande
plt.plot(
    trayectoria_x[0], trayectoria_y[0],
    'go', markersize=15,
    label='Inicio',
    markeredgecolor='black',
    markeredgewidth=2
)

# Marcar final con estrella amarilla
plt.plot(
    trayectoria_x[-1], trayectoria_y[-1],
    'y*', markersize=20,
    label='Final',
    markeredgecolor='black',
    markeredgewidth=2
)

# === Agregar flechas para mostrar direccion del movimiento ===
# Solo dibujamos cada 3ra flecha para no saturar la visualizacion
for i in range(1, len(trayectoria_x) - 1, 3):
    # Calcular componentes del vector direccion (escalado al 50%)
    dx = (trayectoria_x[i] - trayectoria_x[i-1]) * 0.5
    dy = (trayectoria_y[i] - trayectoria_y[i-1]) * 0.5

    # Dibujar flecha desde posicion anterior
    plt.arrow(
        trayectoria_x[i-1], trayectoria_y[i-1],
        dx, dy,
        color='black',
        head_width=0.15,
        head_length=0.15,
        zorder=10
    )

# === Configuracion final ===
plt.xlabel('X', fontsize=12)
plt.ylabel('Y', fontsize=12)
plt.title(f'Trayectoria del compositor {compositor_id} sobre Alpine',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Veamos una animación

In [ ]:
import matplotlib.animation as animation
from IPython.display import Video  # Para visualizar en Colab

In [ ]:
# === Crear malla para curvas de nivel ===
x_vals = np.linspace(-5.0, 5.0, 100)
y_vals = np.linspace(-5.0, 5.0, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = alpine(X, Y)

# === Configurar figura y ejes ===
fig, ax = plt.subplots(figsize=(8, 7))

# Graficar curvas de nivel
contour = ax.contourf(X, Y, Z, levels=30, cmap='viridis')
ax.contour(X, Y, Z, levels=20, colors='black', alpha=0.3)

# Agregar barra de color
plt.colorbar(contour, ax=ax, label='Valor de Alpine')

# Configurar limites y etiquetas
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('Evolucion de las melodías de MMC', fontsize=14, fontweight='bold')

# === Inicializar circulos para cada individuo ===
circles = [
    plt.Circle((coord_x[0][i], coord_y[0][i]), 0.15,
               color='blue', zorder=5)
    for i in range(num_compositores)
]

# Agregar circulos al grafico
for circle in circles:
    ax.add_artist(circle)

# === Funcion de actualizacion para cada frame ===
def actualizar_frame(frame):
    """Actualiza la posicion de cada individuo en la generacion dada."""
    for i, circle in enumerate(circles):
        # Actualizar centro del circulo a la nueva posicion
        nueva_posicion = (coord_x[frame][i], coord_y[frame][i])
        circle.set_center(nueva_posicion)

    return circles
# === Crear animacion ===
animacion = animation.FuncAnimation(
    fig,
    actualizar_frame,
    frames=len(coord_x),
    interval=500,  # 500 ms entre frames (medio segundo)
    blit=True,
    repeat=True
)

# === Guardar como archivo MP4 ===
animacion.save(
    'ed_animacion.mp4',
    writer='ffmpeg',
    fps=2,  # 2 frames por segundo
    dpi=100
)

print("Animacion guardada como: ed_animacion.mp4")

# === Visualizar en Google Colab ===
Video('ed_animacion.mp4', embed=True)